In [18]:
from pyspark.sql import SparkSession

# Intialize Pyspark
spark = SparkSession.builder.appName("LoadHaiTrainData").getOrCreate()


In [19]:
# load training dataset
df1 = spark.read.csv(r"C:\Users\irenm\Desktop\hai-train\hai-train1.txt", header=True, inferSchema=True)
df2 = spark.read.csv(r"C:\Users\irenm\Desktop\hai-train\hai-train2.txt", header=True, inferSchema=True)
df3 = spark.read.csv(r"C:\Users\irenm\Desktop\hai-train\hai-train3.txt", header=True, inferSchema=True)
df4 = spark.read.csv(r"C:\Users\irenm\Desktop\hai-train\hai-train4.txt", header=True, inferSchema=True)

# combine DataFrames
combined_df = df1.union(df2).union(df3).union(df4)


In [20]:
# Print Schema to check column names and data types
combined_df.printSchema()

# Calculate descriptive statistics to check the distribution of numeric columns
combined_df.describe().show()


root
 |-- timestamp: timestamp (nullable = true)
 |-- P1_FCV01D: double (nullable = true)
 |-- P1_FCV01Z: double (nullable = true)
 |-- P1_FCV02D: double (nullable = true)
 |-- P1_FCV02Z: double (nullable = true)
 |-- P1_FCV03D: double (nullable = true)
 |-- P1_FCV03Z: double (nullable = true)
 |-- P1_FT01: double (nullable = true)
 |-- P1_FT01Z: double (nullable = true)
 |-- P1_FT02: double (nullable = true)
 |-- P1_FT02Z: double (nullable = true)
 |-- P1_FT03: double (nullable = true)
 |-- P1_FT03Z: double (nullable = true)
 |-- P1_LCV01D: double (nullable = true)
 |-- P1_LCV01Z: double (nullable = true)
 |-- P1_LIT01: double (nullable = true)
 |-- P1_PCV01D: double (nullable = true)
 |-- P1_PCV01Z: double (nullable = true)
 |-- P1_PCV02D: double (nullable = true)
 |-- P1_PCV02Z: double (nullable = true)
 |-- P1_PIT01: double (nullable = true)
 |-- P1_PIT01_HH: integer (nullable = true)
 |-- P1_PIT02: double (nullable = true)
 |-- P1_PP01AD: integer (nullable = true)
 |-- P1_PP01AR: 

In [21]:
# delete the row of NaN
combined_df = combined_df.na.drop()


In [22]:
from pyspark.sql.functions import to_timestamp, hour, minute


combined_df = combined_df.withColumn('timestamp', to_timestamp(combined_df['timestamp'], 'yyyy-MM-dd HH:mm:ss'))

# Extract hours and minutes from the timestamp column
combined_df = combined_df.withColumn('hour', hour(combined_df['timestamp']))
combined_df = combined_df.withColumn('minute', minute(combined_df['timestamp']))




In [23]:
# Show DataFrame
combined_df.show()


+-------------------+---------+---------+---------+---------+---------+---------+--------+---------+----------+----------+---------+---------+---------+---------+---------+---------+---------+---------+---------+--------+-----------+--------+---------+---------+---------+---------+--------+--------+--------+--------+---------+---------+---------+-------+--------+--------+--------+--------+------------+---------+---------+--------+-------+------------+-----------+-----------+--------+------+------+---------+--------+---------+----------+----------+----------+----------+--------+--------+--------+--------+--------+--------+---------+---------+-------+--------+-------+--------+--------+--------+--------+---------+--------+---------+---------+---------+--------+----------+----------+---------------------+-------------------+---------------------+---------------------+---------------------+---------------------+----------------+----+------+
|          timestamp|P1_FCV01D|P1_FCV01Z|P1_FCV02D

In [24]:
from pyspark.sql.functions import to_timestamp, window, col, avg

# Ensure timestamps are formatted correctly
combined_df = combined_df.withColumn('timestamp', to_timestamp(col('timestamp'), 'yyyy-MM-dd HH:mm:ss'))

# Sort data by timestamp
combined_df = combined_df.orderBy('timestamp')

# Resampling and aggregation of data, e.g., hourly averages
hourly_avg_df = combined_df.groupBy(window(col('timestamp'), "1 hour")).avg()


In [25]:
hourly_avg_df.show()

+--------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+--------------+--------------+--------------+--------------+-------------+-------------+------------------+-------------------+-----------------+--------------+--------------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------+-------------------+-------------------+-------------------+------------------+-------------+-----------+------------------+-----------------+-----------------+--------------+------------------+-------------------+-------------

In [26]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x000001EB6887A100>>